In [32]:
from ase.io import iread
import numpy as np
from ase.atoms import Atoms
from pymatgen.core import Composition
from collections import Counter
from math import gcd
from functools import reduce 
from ase.data import atomic_numbers
from ase.visualize import view

# extxyz 파일 읽기
file_path = "/home/minkyu/DiffCSP/oc20_s2ef_2M/train/extxyz/0.extxyz"

# 첫 번째 프레임만 읽어서 분석
# atoms = next(iread(file_path))
atoms_iter = iread(file_path)
# next(atoms_iter)  # 첫 번째 프레임 건너뛰기
atoms = next(atoms_iter)  # 두 번째 프레임 읽기

print("=== 기본 구조 정보 ===")
print(f"원자 수: {len(atoms)}")
print(f"화학식: {atoms.get_chemical_formula()}")
print(f"원자 번호: {atoms.get_atomic_numbers()}")
print(f"원소 종류: {set(atoms.get_chemical_symbols())}")

print("\n=== 셀 정보 ===")
cell = atoms.get_cell()
print(f"셀 벡터:\n{cell}")
print(f"셀 부피: {atoms.get_volume():.6f} Å³")
print(f"주기 경계 조건: {atoms.get_pbc()}")

# print("\n=== 원자 위치 (처음 5개) ===")
# positions = atoms.get_positions()
# for i, pos in enumerate(positions[:5]):
#     symbol = atoms.get_chemical_symbols()[i]
#     print(f"{symbol} {i}: ({pos[0]:.6f}, {pos[1]:.6f}, {pos[2]:.6f})")

print("\n=== 추가 속성들 ===")
print(f"사용 가능한 배열: {list(atoms.arrays.keys())}")

# Tags 정보
if 'tags' in atoms.arrays:
    tags = atoms.get_array('tags')
    unique_tags = np.unique(tags)
    print(f"\nTags: {unique_tags}")
    for tag in unique_tags:
        count = np.sum(tags == tag)
        print(f"Tag {tag}: {count}개 원자")

mask = tags != 2
slab = atoms[mask]

slab_atomic_numbers = slab.get_atomic_numbers()

# 원소별 개수 계산
element_counts = {}
for atomic_num in slab_atomic_numbers:
    element_counts[atomic_num] = element_counts.get(atomic_num, 0) + 1

# 최소 공약수로 나누어 최소 비율 구하기
counts = list(element_counts.values())
if counts:
    from math import gcd
    from functools import reduce
    gcd_value = reduce(gcd, counts)
    
    # 최소 비율 계산
    min_ratios = {k: v // gcd_value for k, v in element_counts.items()}
    
    # Composition: [[원소번호들], [비율들]] 형태
    elements = list(min_ratios.keys())
    ratios = list(min_ratios.values())
    total_ratio = sum(ratios)
    normalized_ratios = [r / total_ratio for r in ratios]
    slab_composition = [elements, normalized_ratios]
    
    # Multiplicity: 배수
    slab_multiplicity = gcd_value
else:
    slab_composition = [[], []]
    slab_multiplicity = 1

# element_counts = {}
# for atomic_num in slab_atomic_numbers:
#     element_counts[atomic_num] = element_counts.get(atomic_num, 0) + 1

# # 최소 공약수로 나누어 최소 비율 구하기
# counts = list(element_counts.values())
# if counts:
#     from math import gcd
#     from functools import reduce
#     gcd_value = reduce(gcd, counts)
    
#     # Composition: 최소 비율
#     slab_composition = {k: v // gcd_value for k, v in element_counts.items()}
    
#     # Multiplicity: 배수
#     slab_multiplicity = gcd_value
# else:
#     slab_composition = {}
#     slab_multiplicity = 1

print("Slab composition:", slab_composition)
print("Slab multiplicity:", slab_multiplicity)

print(atoms.get_array('tags'))

view(atoms, viewer='ngl')


=== 기본 구조 정보 ===
원자 수: 71
화학식: C2H4Ag16Ca32Cu16O
원자 번호: [20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20
 20 20 20 20 20 20 20 20 29 29 29 29 29 29 29 29 29 29 29 29 29 29 29 29
 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47 47  6  6  1  1  1  1  8]
원소 종류: {'C', 'H', 'Ca', 'O', 'Ag', 'Cu'}

=== 셀 정보 ===
셀 벡터:
Cell([[9.18446134, 0.0, 0.0], [0.0, 22.63986204, -4.15391219], [0.0, 0.0, 29.07738533]])
셀 부피: 6046.204306 Å³
주기 경계 조건: [ True  True  True]

=== 추가 속성들 ===
사용 가능한 배열: ['numbers', 'positions', 'tags']

Tags: [0 1 2]
Tag 0: 48개 원자
Tag 1: 16개 원자
Tag 2: 7개 원자
Slab composition: [[20, 29, 47], [0.5, 0.25, 0.25]]
Slab multiplicity: 16
[0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0
 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 2 2 2 2 2 2 2]


In [ ]:
import pandas as pd
import ast
import numpy as np

# CSV 파일에서 atom_types 컬럼만 읽기
df = pd.read_csv(
    '/home/minkyu/DiffCSP/train.csv', 
    usecols=['atom_types']
)

print(f"데이터셋 크기: {len(df)}")
print(f"atom_types 컬럼 샘플:")
print(df['atom_types'].head())

# 데이터 타입 확인
print(f"\natom_types 데이터 타입: {type(df['atom_types'].iloc[0])}")
print(f"첫 번째 샘플의 내용: {df['atom_types'].iloc[0]}")

# atom_types에서 모든 atomic number들을 추출하는 함수
def extract_atomic_numbers(atom_types_data):
    try:
        # numpy 배열인 경우
        if isinstance(atom_types_data, np.ndarray):
            return atom_types_data.tolist()
        # 문자열인 경우 (numpy 배열이 문자열로 저장된 경우)
        elif isinstance(atom_types_data, str):
            # 문자열에서 numpy 배열 형태를 파싱
            if atom_types_data.startswith('[') and atom_types_data.endswith(']'):
                # 대괄호로 둘러싸인 경우
                atom_types_data = atom_types_data.strip('[]')
            # 공백으로 구분된 숫자들을 분리
            numbers = []
            for num_str in atom_types_data.split():
                try:
                    numbers.append(int(num_str))
                except ValueError:
                    continue
            return numbers
        # 리스트인 경우
        elif isinstance(atom_types_data, list):
            return atom_types_data
        else:
            return []
    except:
        return []

# 모든 atomic number들을 수집
all_atomic_numbers = []
print("\n처리 중...")
for i, atom_types_data in enumerate(df['atom_types']):
    if i % 100000 == 0:
        print(f"진행률: {i/len(df)*100:.1f}%")
    
    atomic_numbers = extract_atomic_numbers(atom_types_data)
    all_atomic_numbers.extend(atomic_numbers)

# 최솟값과 최댓값 계산
if all_atomic_numbers:
    min_atomic_number = min(all_atomic_numbers)
    max_atomic_number = max(all_atomic_numbers)
    
    print(f"\n=== 결과 ===")
    print(f"atom_types에 포함된 atomic number들의 최솟값: {min_atomic_number}")
    print(f"atom_types에 포함된 atomic number들의 최댓값: {max_atomic_number}")
    
    # 추가 통계 정보
    unique_atomic_numbers = sorted(set(all_atomic_numbers))
    print(f"\n고유한 atomic number 개수: {len(unique_atomic_numbers)}")
    print(f"고유한 atomic number들: {unique_atomic_numbers}")
    
    # 각 원소의 빈도수 계산
    from collections import Counter
    atomic_counter = Counter(all_atomic_numbers)
    print(f"\n가장 많이 나타나는 atomic number들 (상위 10개):")
    for atomic_num, count in atomic_counter.most_common(10):
        print(f"  원소 번호 {atomic_num}: {count}회")
    
else:
    print("atom_types 데이터에서 atomic number를 추출할 수 없습니다.") 

In [ ]:
# slab_composition 데이터를 파싱하여 첫 번째 원소의 길이 계산
def get_first_element_length(composition_str):
    try:
        # 문자열을 리스트로 파싱
        composition = ast.literal_eval(composition_str)
        if isinstance(composition, list) and len(composition) > 0:
            return len(composition[0])  # 첫 번째 원소의 길이
        else:
            return 0
    except:
        return 0

# 각 slab_composition[0]의 길이 계산
lengths = df['slab_composition'].apply(get_first_element_length)

# 최댓값과 최솟값 계산
min_length = lengths.min()
max_length = lengths.max()

print(f"slab_composition[0] 길이의 최솟값: {min_length}")
print(f"slab_composition[0] 길이의 최댓값: {max_length}")

# 길이별 분포 확인
print(f"\n길이별 분포:")
print(lengths.value_counts().sort_index())

# 최솟값과 최댓값을 가지는 데이터 예시
print(f"\n최솟값({min_length})을 가지는 데이터 예시:")
min_examples = df[lengths == min_length]['slab_composition'].head(3)
for i, example in enumerate(min_examples):
    print(f"  {i+1}: {example}")

print(f"\n최댓값({max_length})을 가지는 데이터 예시:")
max_examples = df[lengths == max_length]['slab_composition'].head(3)
for i, example in enumerate(max_examples):
    print(f"  {i+1}: {example}")

In [23]:
# CIF 파일을 ASE view로 시각화
from ase.io import read
from ase.visualize import view

# CIF 파일 읽기
cif_path = "/home/minkyu/DiffCSP/hydra/singlerun/2025-09-16/no_shuffle_soft_label/eval_diff.dir/0/cif/30.cif"
atoms = read(cif_path)

print("=== CIF 파일 정보 ===")
print(f"원자 수: {len(atoms)}")
print(f"화학식: {atoms.get_chemical_formula()}")
print(f"셀 부피: {atoms.get_volume():.6f} Å³")
print(f"셀 벡터: {atoms.cell}")

# ASE view로 시각화 (3D 뷰어가 열림)
view(atoms, viewer='ngl')


=== CIF 파일 정보 ===
원자 수: 84
화학식: C2H2N53W27
셀 부피: 4336.002666 Å³
셀 벡터: Cell([[11.47514343, 0.0, 0.0], [-0.2593226750727832, 13.388130917912903, 0.0], [-1.4723125906598282, 11.351981496938055, 28.223538850535753]])


In [1]:
from ase.io import iread
import numpy as np
from ase.atoms import Atoms
from pymatgen.core import Composition
from collections import Counter
from math import gcd
from functools import reduce 
from ase.data import atomic_numbers

# extxyz 파일 읽기
file_path = "/home/minkyu/DiffCSP/oc20_s2ef_2M/train/extxyz/0.extxyz"

# 첫 번째 프레임만 읽어서 분석
# atoms = next(iread(file_path))
atoms_iter = iread(file_path)
# next(atoms_iter)  # 첫 번째 프레임 건너뛰기
atoms = next(atoms_iter)  # 두 번째 프레임 읽기

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from ase import Atoms

def check_slab_can_be_reduced_with_symmetry(atoms: Atoms, tol=0.01):
    # ASE -> Pymatgen 변환
    lattice = atoms.get_cell()
    species = [atom.symbol for atom in atoms]
    coords = atoms.get_scaled_positions()
    structure = Structure(lattice, species, coords, coords_are_cartesian=False)

    sga = SpacegroupAnalyzer(structure, symprec=tol)
    primitive = sga.find_primitive()
    
    return primitive.num_sites < len(structure)

# 전체 카운트 초기화
can_reduce_count = 0
already_unit_count = 0

for atoms in atoms_iter:
    if check_slab_can_be_reduced_with_symmetry(atoms):
        can_reduce_count += 1
    else:
        already_unit_count += 1

print(f"총 {can_reduce_count + already_unit_count}개의 frame 중")
print(f"{can_reduce_count}개는 더 작은 unit cell로 줄일 수 있음")
print(f"{already_unit_count}개는 이미 최소 unit cell")

총 4999개의 frame 중
0개는 더 작은 unit cell로 줄일 수 있음
4999개는 이미 최소 unit cell


In [6]:
import numpy as np
from ase import Atoms
from ase.io import iread
import numpy as np
from ase.atoms import Atoms
from pymatgen.core import Composition
from collections import Counter
from math import gcd
from functools import reduce 
from ase.data import atomic_numbers
from ase.io import write

# extxyz 파일 읽기
file_path = "/home/minkyu/DiffCSP/oc20_s2ef_2M/train/extxyz/0.extxyz"

# 첫 번째 프레임만 읽어서 분석
# atoms = next(iread(file_path))
atoms_iter = iread(file_path)
# next(atoms_iter)  # 첫 번째 프레임 건너뛰기
atoms = next(atoms_iter)  # 두 번째 프레임 읽기

def center_adsorbate_xy_only(atoms: Atoms, adsorbate_indices):
    """
    atoms: ASE Atoms 객체 (unit cell 기준)
    adsorbate_indices: unit cell 내 adsorbate 원자의 인덱스 리스트

    returns: 새로운 Atoms 객체 (x, y만 중앙화, z는 그대로)
    """
    # 1. adsorbate fractional 좌표 추출
    ads_frac = atoms.get_scaled_positions()[adsorbate_indices]

    # 2. adsorbate 중심 계산 (x, y만)
    center_xy = np.mean(ads_frac[:, :2], axis=0)

    # 3. unit cell 중심으로 이동 (x, y = 0.5, 0.5)
    shift = np.array([0.5, 0.5]) - center_xy

    # 4. 전체 atoms에 shift 적용 (x, y만), z는 그대로
    new_atoms = atoms.copy()
    new_frac = new_atoms.get_scaled_positions()
    new_frac[:, 0] = (new_frac[:, 0] + shift[0]) % 1.0  # x wrap
    new_frac[:, 1] = (new_frac[:, 1] + shift[1]) % 1.0  # y wrap
    # z는 변경 없음
    new_atoms.set_scaled_positions(new_frac)

    return new_atoms

# 사용 예시:
centered_atoms = center_adsorbate_xy_only(atoms, adsorbate_indices=atoms.get_array("tags")==2)

write("original_atoms.cif", atoms)
write("centered_atoms.cif", centered_atoms)

In [3]:
from ase.visualize import view
view(centered_atoms, viewer="ngl")

/home/minkyu/micromamba/envs/slabflow/lib/python3.9/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
import os
from ase.io import iread
from tqdm import tqdm

extxyz_dir = "/home/minkyu/DiffCSP/oc20_s2ef_2M/val_id/extxyz/"
k_values = []

for fname in tqdm(os.listdir(extxyz_dir)):
    if not fname.endswith(".extxyz"):
        continue
    fpath = os.path.join(extxyz_dir, fname)
    for atoms in iread(fpath):
        tags = atoms.get_array("tags")
        adsorbate_indices = (tags == 2)
        adsorbate_atomic_numbers = atoms.get_atomic_numbers()[adsorbate_indices]
        k = len(adsorbate_atomic_numbers)
        k_values.append(k)

print("최솟값:", min(k_values))
print("최댓값:", max(k_values))

100%|██████████| 200/200 [09:09<00:00,  2.75s/it]

최솟값: 1
최댓값: 11


In [15]:
import pandas as pd
import ast
import numpy as np
from ase import Atoms
from ase.visualize import view

# # CSV 파일에서 필요한 컬럼들 읽기
# df = pd.read_csv(
#     '/home/minkyu/DiffCSP/val.csv',
# )

# import pandas as pd

# val.csv 파일의 첫 번째 row만 읽기
df_first_row = pd.read_csv('/home/minkyu/DiffCSP/val.csv', nrows=1)

# 첫 번째 row를 새로운 CSV 파일로 저장
df_first_row.to_csv('/home/minkyu/DiffCSP/val_first_row.csv', index=False)

print("첫 번째 row가 val_first_row.csv로 저장되었습니다.")
print(f"저장된 데이터:\n{df_first_row}")

idx = 0 

# 문자열로 저장된 데이터를 파이썬 객체로 변환
atom_types = ast.literal_eval(df.loc[idx, 'atom_types'])
lengths = ast.literal_eval(df.loc[idx, 'lengths'])
angles = ast.literal_eval(df.loc[idx, 'angles'])
frac_coords = ast.literal_eval(df.loc[idx, 'frac_coords'])

# cell 파라미터 결합 (lengths + angles)
cell = lengths + angles

print(f"원자 종류: {atom_types}")
print(f"셀 길이: {lengths}")
print(f"셀 각도: {angles}")
print(f"셀 파라미터: {cell}")
print(f"분수 좌표 개수: {len(frac_coords)}")

# ASE Atoms 객체 생성
atoms = Atoms(symbols=atom_types, scaled_positions=frac_coords, cell=cell, pbc=True)

print(f"생성된 구조 정보:")
print(f"  - 원자 수: {len(atoms)}")
print(f"  - 화학식: {atoms.get_chemical_formula()}")
print(f"  - 셀 부피: {atoms.get_volume():.2f} Å³")

# 3D 시각화
view(atoms, viewer='ngl')

첫 번째 row가 val_first_row.csv로 저장되었습니다.
저장된 데이터:
   Unnamed: 0      system_id  \
0           0  random2163697   

                                          atom_types  \
0  [82, 82, 82, 82, 82, 82, 82, 82, 34, 34, 34, 3...   

                                         lengths  \
0  [8.77403983, 15.191063301410269, 28.76681856]   

                            angles  \
0  [95.06533765919089, 90.0, 90.0]   

                                         frac_coords    energy  num_atoms  \
0  [[0.47506714290240465, 0.21975986898030336, 0.... -155.9246         40   

   spacegroup.number                   slab_composition  ... slab_num_atoms  \
0                  1  [[82, 34, 50], [0.25, 0.5, 0.25]]  ...             32   

  slab_multiplicity                                    slab_atom_types  \
0                 8  [82, 82, 82, 82, 82, 82, 82, 82, 34, 34, 34, 3...   

                                    slab_frac_coords  \
0  [[0.47506714290240465, 0.21975986898030336, 0....   

            adsor

In [12]:
idx = 9

# 문자열로 저장된 데이터를 파이썬 객체로 변환
atom_types = ast.literal_eval(df.loc[idx, 'atom_types'])
lengths = ast.literal_eval(df.loc[idx, 'lengths'])
angles = ast.literal_eval(df.loc[idx, 'angles'])
frac_coords = ast.literal_eval(df.loc[idx, 'frac_coords'])
cell = lengths + angles
# ASE Atoms 객체 생성
atoms = Atoms(symbols=atom_types, scaled_positions=frac_coords, cell=cell, pbc=True)

# 3D 시각화
# view(atoms, viewer='ngl')
print(atom_types)

[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 6, 6, 1, 1, 1, 1, 8]


In [11]:
print(atom_types)

[40, 40, 40, 40, 40, 40, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 42, 42, 42, 42, 42, 42, 8, 1]


In [14]:
first_row_df = df.loc[[0]]  # DataFrame 형태로 유지
first_row_df.to_csv('first_row_data.csv', index=False)

In [1]:
import pandas as pd

df = pd.read_csv('data/oc20_2M/val.csv')
sample_df = df.sample(n=100, random_state=42)  # random_state로 재현성 확보
sample_df.to_csv('test.csv', index=False)